In [9]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [10]:
from bs4 import BeautifulSoup
import requests

In [11]:
def crawl_dbp(word):
    url = f'https://prpm.dbp.gov.my/Cari1?keyword={word}'
    r = requests.get(url)
    soup = BeautifulSoup(r._content)
    definitions = soup.select("div[class*=tab-pane]")
    definitions = [d.text for d in definitions]
    spans = soup.find_all('span', id = 'MainContent_SearchInfoTesaurus_lblTesaurus')
    if len(spans):
        tesaurus = spans[0]
        tds = tesaurus.findAll('td')
        selected_td = None
        for td in tds:
            if 'javascript:showModalDialog' in str(td):
                selected_td = td
        if selected_td:
            tesaurus = [a.text for a in selected_td.find_all('a')]
        else:
            tesaurus = None
    else:
        tesaurus = None
    return {'definisi': definitions, 'tesaurus': tesaurus}

In [6]:
import malaya

In [23]:
%%time

crawl_dbp('jaja')

CPU times: user 25.2 ms, sys: 0 ns, total: 25.2 ms
Wall time: 432 ms


{'definisi': ['[ja.ja]  |  جاجDefinisi : ; berjaja 1. mencari rezeki (nafkah, pen\xaddapatan, dsb) dgn membawa barang dagangan berkeliling utk dijual: Yahya didenda kerana ~ tanpa lesen; 2. berjual: dia dilarang ~ di situ; menjajakan 1. menjual barang dagangan dgn membawanya berkeliling: riuh-rendah bunyi orang ~ barang dagangan masing-masing; 2. ki menyebarkan (menyatakan dsb) ke sana sini atau merata-rata: Menteri Pen\xaddidikan menasihatkan rakyat supaya tidak terperangkap dgn fikiran sempit go\xadlongan yg ~ pendapat mereka sbg sesuatu yg mutlak; ~ khabar; jajaan benda atau barang yg dijajakan; penjajaan perbuatan (kerja) menjajakan; \xadpenjaja orang yg berjaja: seorang ~ telah di\xadkenakan denda sebanyak RM100.\xa0(Kamus Dewan Edisi Keempat)',
  '[ja.ja]  |  جاجDefinisi : ; berjaja membawa barang-barang dagangan ke merata-rata tempat utk dijual: Tiap-tiap pagi, Jalal ~ nasi lemak. menjajakan menjual sesuatu dgn membawanya berkeliling: Ia pergi dr rumah ke rumah utk ~ hasil kebun

In [5]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [12]:
import mp
import os
import json
from tqdm import tqdm
import traceback
import redis

In [13]:
db = redis.Redis(host='localhost', port=6379, db=11)

In [49]:
len(db.keys())

187385

In [45]:
db.keys()

[b'basiasi',
 b'dijaja',
 b'sirna',
 b'mesovarium',
 b'videogram',
 b'geledur',
 b'wuzzer',
 b'sina',
 b'nosokomial',
 b'guaycuruan']

In [16]:
db['basiasi']

b'{"definisi": [], "tesaurus": null}'

In [17]:
def loop(words):
    words, _ = words
    db = redis.Redis(host='localhost', port=6379, db=11)
    for word in tqdm(words):
        if word in db:
            continue
        try:
            r = crawl_dbp(word)
            db.set(word, json.dumps(r))
        except Exception as e:
            print(e)
            # print(traceback.format_exc())

In [31]:
with open('malay-words-v2.json') as fopen:
    words = json.load(fopen)
    
len(words)

187385

In [32]:
words[:10]

['sina',
 'guaycuruan',
 'mesovarium',
 'nosokomial',
 'sirna',
 'myoxine',
 'requarantine',
 'vomerobasilar',
 'hydroterpene',
 'embriematis']

In [21]:
stemmer = malaya.stem.deep_model(model = 'noisy')

2022-09-16 21:07:55.375734: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 21:07:55.400161: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-16 21:07:55.400181: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-09-16 21:07:55.400185: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-09-16 21:07:55.400247: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-09-16 21:07:55.400264: I

In [33]:
stemmer.stem_word

<bound method DeepStemmer.stem_word of <malaya.stem.DeepStemmer object at 0x7f6e2fbeaa30>>

In [35]:
from tqdm import tqdm

stemmed_words = []
for w in tqdm(words):
    stemmed = stemmer.stem_word(w)
    stemmed_words.append(stemmed)

 82%|███████████████████████████████████████████████████████████████████▌              | 154368/187385 [14:42<03:17, 167.14it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
len(stemmed_words)

187385

In [42]:
new_words = set(stemmed_words) | set(words)
len(new_words)

191200

In [43]:
loop((words[:5], 0),)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.05it/s]


In [ ]:
mp.multiprocessing(words, loop, cores = 50, returned = False)

 12%|██████████▍                                                                           | 456/3747 [58:09<5:29:29,  6.01s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

